# WRANGLING DATA WITH TWITTER API AND TWEEPY
### COMPARE THE TWEETS FROM MY GERMAN FRIENDS/FOLLOWERS WITH MY AMERICAN FRIENDS/FOLLOWERS

# Step 0:
### Importing libraries and authorizing Twitter API:

In [1]:
# import sys
# !{sys.executable} -m pip install tweepy
import tweepy
import json
import tweepy
import pandas as pd

In [2]:
# This doesnt work (taken from class notes to refresh memory)

# cred = {}
# cred['API_KEY'] = API_KEY
# cred['API_SECRET'] = API_SECRET
# cred['ACCESS_TOKEN'] = ACCESS_TOKEN
# cred['ACCESS_TOKEN_SECRET'] = ACCESS_TOKEN_SECRET

# with open('twitter_credentials copy2.json', 'w') as outfile:
#     json.dump(cred, outfile)

In [3]:
#note: in case this doesnt work, I changed the name of the file (before it was "twitter_credentials copy.json") to add to gitignore
with open('twitter_credentials_copy.json', 'r') as infile:
    credentials = json.loads(infile.read())

#### Twitter API - documentation:
https://docs.tweepy.org/en/latest/api.html 

In [4]:
help(tweepy)

# need to create an authentication object to authenticate ourselves!

Help on package tweepy:

NAME
    tweepy - Tweepy Twitter API library

PACKAGE CONTENTS
    api
    auth
    binder
    cache
    cursor
    error
    models
    parsers
    streaming
    utils

FUNCTIONS
    debug(enable=True, level=1)

DATA
    __license__ = 'MIT'
    api = <tweepy.api.API object>

VERSION
    3.10.0

AUTHOR
    Joshua Roesslein

FILE
    /opt/anaconda3/lib/python3.8/site-packages/tweepy/__init__.py




In [5]:
auth = tweepy.OAuthHandler(credentials['API_KEY'], credentials['API_SECRET'])

# step 1: authenticate yourself
# this can be found in the authentication tutorial on the twitter developer page docs.

In [6]:
# step 2: set your tokens
# we need to now SET our access tokens
# this sets the tokens for you to actually be able to receive information from the API

auth.set_access_token(credentials['ACCESS_TOKEN'], credentials['ACCESS_TOKEN_SECRET'])

In [7]:
# step 3: generate your API
# now create our access to the API

api = tweepy.API(auth)

In [8]:
api.me()

# when working with complex APIs that require authentication, it is good to run a simple command like this at the very beginning
# so you can be sure that the authentication worked! 

User(_api=<tweepy.api.API object at 0x1069e4fa0>, _json={'id': 979279794478206976, 'id_str': '979279794478206976', 'name': 'Kate Saslow', 'screen_name': 'kate_saslow', 'location': 'Berlin, Germany', 'profile_location': {'id': '3078869807f9dd36', 'url': 'https://api.twitter.com/1.1/geo/id/3078869807f9dd36.json', 'place_type': 'unknown', 'name': 'Berlin, Germany', 'full_name': 'Berlin, Germany', 'country_code': '', 'country': '', 'contained_within': [], 'bounding_box': None, 'attributes': {}}, 'description': 'Research analyst and avid reader on all things #AI, #datascience, #feminism, #policy. All views my own. w/ @snv_berlin. Alumna of @thehertieschool @UMich.', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 589, 'friends_count': 963, 'listed_count': 15, 'created_at': 'Thu Mar 29 08:51:11 +0000 2018', 'favourites_count': 358, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 183, 'lang': None

# Step 1:
### Gather Friends and Followers to later gather their tweets:

#### Filter through my Twitter followers by their location:
https://underthecurve.github.io/jekyll/update/2017/08/02/twitter-followers.html

In [11]:
# api.followers?

In [12]:
# users = [user for user in api.followers('id', 'screen_name', count=250)]
# users

#### Make a dataframe with my Friends:

In [13]:
# function from SNV twitter network:

def get_friends(user):
    """ returns a list of friends (twitter_handle) of a user"""
    friends = []
    n=0
    for page in tweepy.Cursor(api.friends, screen_name=user, count=200).pages():
        n+=1
        print(f"Page number {n}")
        friends.extend(page)
#       time.sleep(60)
        return (friends)

kate_friends = get_friends("kate_saslow")

Page number 1


In [ ]:
kate_friends[0]._json

In [15]:
names_friends = [kate_friends[i]._json['name'] for i in range(len(kate_friends))]

In [31]:
handles_friends = [kate_friends[i]._json['screen_name'] for i in range(len(kate_friends))]

In [32]:
locations_friends = [kate_friends[i]._json['location'] for i in range(len(kate_friends))]

In [33]:
descriptions_friends = [kate_friends[i]._json['description'] for i in range(len(kate_friends))]

In [34]:
friends_df = pd.DataFrame(zip(names_friends, handles_friends, locations_friends, descriptions_friends))
friends_df.rename(columns={0:'Friend', 1:'Handle', 2:'Location', 3:'UserDescription'}, inplace=True)
friends_df

,Friend,Handle,Location,UserDescription
0,Treva B,divafeminist,All Ovah the World,Prof @ OSU/feminist/dancer/friend/daughter/aun...
1,Veronika,VeronikaDatzer,Berlin,"International Relations @FU_Berlin, Assistant ..."
2,Christina Rupp,christinacrupp,"Bonn, Deutschland",Project Assistant International Cybersecurity ...
3,Clara,thatclarafied,"Los Angeles, CA",Writer | Comedian | Tech | Diversity & Inclusi...
4,John Miller,JohnMillerTX,"Fort Worth, TX","#Analytics leader dealing in #machinelearning,..."
...,...,...,...,...
195,Esther,es_kern,Germany,🇪🇺 Research Fellow @BIGS_Potsdam | @Polis180 |...
196,Anna Cardillo,annacardillo_,Berlin,Datenschutz-|Informationssicherheitsrecht|Busi...
197,Nils Leopold,NilsLeopold,"Berlin, Germany",Berlin
198,Bart Hoogeveen,BartHoogeveen,Canberra,🇪🇺 with @ASPI_ICPC leading on #CyberCapacityBu...


In [35]:
# save friends_df to csv for further analysis:

friends_df.to_csv('twitter_friends_info_df.csv')

In [36]:
len(friends_df['Handle'])

# why are there only 20 friends? 

200

#### Make dataframe with my Followers:

In [37]:
# function from twitter SNV analysis:

def get_follower(user):
    '''Returns a list of all follower (twitter_handle) of a user
    taking into account twitter API's request frequency limits'''
    follower = []
    n=0
    for page in tweepy.Cursor(api.followers, screen_name=user, count=200).pages():
        n+=1
        print(f"Page number {n}")
        follower.extend(page)
#       time.sleep(60)
    #screen_names = [user.screen_name for user in friends]
    return (follower)

kate = get_follower('kate_saslow')
kate[0]

Page number 1
Page number 2
Page number 3


User(_api=<tweepy.api.API object at 0x121cc70d0>, _json={'id': 989902718637608961, 'id_str': '989902718637608961', 'name': 'Christina Rupp', 'screen_name': 'christinacrupp', 'location': 'Bonn, Deutschland', 'description': 'Project Assistant International Cybersecurity Policy @snv_berlin', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 25, 'friends_count': 193, 'listed_count': 0, 'created_at': 'Fri Apr 27 16:22:54 +0000 2018', 'favourites_count': 4, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 1, 'lang': None, 'status': {'created_at': 'Thu Dec 17 19:55:23 +0000 2020', 'id': 1339660483541000193, 'id_str': '1339660483541000193', 'text': 'RT @z_edian: Unser @snv_berlin Expert:innenworkshop zur "staatlichen Beantwortung von böswilligen Cyberaktivitäten" geht langsam zuende.\n\nD…', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'z_edian', 'n

In [38]:
type(kate[0]._json)

dict

In [ ]:
kate[0]._json

In [40]:
names_followers = [kate[i]._json['name'] for i in range(len(kate))]

In [41]:
handles_followers = [kate[i]._json['screen_name'] for i in range(len(kate))]

In [42]:
locations_followers = [kate[i]._json['location'] for i in range(len(kate))]

In [43]:
descriptions_followers = [kate[i]._json['description'] for i in range(len(kate))]

In [71]:
followers_df = pd.DataFrame(zip(names_followers,handles_followers, locations_followers, descriptions_followers))
followers_df.rename(columns={0:'Follower', 1:'Handle', 2:'Location', 3:'UserDescription'}, inplace=True)
followers_df

# will eventually need to clean to standardize how the location is formatted (e.g. by country only)

,Follower,Handle,Location,UserDescription
0,Christina Rupp,christinacrupp,"Bonn, Deutschland",Project Assistant International Cybersecurity ...
1,Johanna Schäfer,joscchh,"Bonn, Deutschland","CEO @BonnLAB, urban developer & #SocialMedia e..."
2,John Miller,JohnMillerTX,"Fort Worth, TX","#Analytics leader dealing in #machinelearning,..."
3,Rosanna Fanni,rosannafanni,,monitoring the European public-political sphere
4,Lottie Lane,lottielane_,"Groningen, Nederland",Assistant Prof. and Postdoc at @univgroningen\...
...,...,...,...,...
582,Stiftung Neue Verantwortung,snv_berlin,Berlin,Der Think Tank Stiftung Neue Verantwortung arb...
583,Sasquatch,SirJacobSaslow,,"Clear eyes, full hearts, cant lose"
584,Nikolas Reschen,n_reschen,"Wien, Österreich","Some sort of Scientist / Labour Market, Educat..."
585,goldklang,goldkl4ng,"Ann Arbor, MI",(he/him) Birds of Detroit: https://t.co/adrMv8...


In [72]:
# Save follower_df to csv:
followers_df.to_csv('twitter_followers_info_df.csv')

# Step 2:
### Gathering and Storing Tweets:

#### Write function that gets tweets by looping through the friends/followers in the list comprehensions above:

In [45]:
def get_tweets(handle):

    # extract 200 tweets
    number_tweets = 200
    tweets = api.user_timeline(screen_name=handle)

    # Empty array
    tmp = []

    # create array of tweet info: username, tweet id, date/time, text
    tweets_for_csv = [tweet.text for tweet in tweets] 

    for j in tweets_for_csv:
        tmp.append(j)
    

    return tmp

In [46]:
# test it to see if it worked, then apply loop to list comp.
get_tweets('SirJacobSaslow')

["RT @BillKristol: Lindsey Graham: A president who doesn't comply with Congressional requests for information is subject to impeachment. http…",
 'RT @DetroitPistons: Reggie Jackson #NBAVote \n\n1 RT = 1 vote! https://t.co/bJSHdjMAW6',
 '#NBAVote Drummond  https://t.co/qkKDCdy69P',
 "RT @espn: That's ten straight wins for the @Tigers. The Motor City's starting to put the AL Central in cruise control.",
 '“@espn: College Football: Approaching fast -&gt; http://t.co/3zAYnhXp26” Espn giving shoutouts to UMICH on the regs #goblue #rosebowlbound',
 '@RapGenius Welcome to Atlanta remix, heeeyyyyy',
 'RT @SeinfeldToday: George is briefly implicated in the latest Anthony Weiner scandal because his OKCupid handle is Carlos Danger.',
 'RT @espn: Miguel Cabrera is the 1st player in major league history with 30 HR and 90 RBI before the All-Star break. #NotoriousMIG',
 'RT @JustinVerlander: #BackBenoit #BackBenoit #BackBenoit #BackBenoit #BackBenoit #BackBenoit',
 'RT @JustinVerlander: RT &amp; hel

#### It seems that extracting the tweets' metadata worked. 
Now try it on all Followers rather than just the first 5!

In [47]:
handles = list(followers_df['Handle'])
handles

['christinacrupp',
 'joscchh',
 'JohnMillerTX',
 'rosannafanni',
 'lottielane_',
 'TobiTheScholz',
 'mnedbrussels',
 'fran_digital',
 'jas_abroad',
 'EgonHuschitt',
 'liradelira0',
 'KamasaJulian',
 'EmiliaNeuber',
 'mattreiner',
 'NinoMacha',
 'E_Dorokhova',
 'sa_zeet',
 'louisjarvers',
 'verenawingerter',
 'r_brendel',
 'machinelearnflx',
 'askans',
 'HeidiObermeyer',
 'GerritW24',
 'ralf_ladner',
 'DennisW16677051',
 'andrea_no_',
 'relationserieu5',
 'Ely_anon_',
 'stormy_mildner',
 'samarabell1818',
 'AlexTuc22110441',
 '_MUNIYANquantum',
 'Egels14',
 'JLMellentin',
 'PolicyIsrael',
 'adahlma',
 'LewinFricke',
 'kat_lampert',
 'SF_Dietlmeier',
 'teamrework',
 'denialstein',
 'tjolp',
 'thehertieschool',
 'MALAKSA55400906',
 'RaquelJorgeR',
 'ForhadulFoysal',
 'Beckswunderland',
 'LordElend',
 'laurajakli',
 'snazzyazzy',
 'HannesMehrer',
 'Tobias__Wolf',
 'BenediktBrecht',
 'Keji94771336',
 'andre_weisser',
 'fedcar89',
 'suyak',
 'spielkamp',
 'brankamarijan',
 'GrinspoonMr',
 'K

In [48]:
handles = list(followers_df['Handle'])
tweets = pd.DataFrame()

for handle in handles[0:5]:
    tweets_handle = pd.DataFrame(get_tweets(handle))
    tweets_handle['Handle'] = handle
    tweets = pd.concat([tweets, tweets_handle], axis=0)
    
tweets.head()

,0,Handle
0,RT @z_edian: Unser @snv_berlin Expert:innenwor...,christinacrupp
0,RT @rachel_groves: Daily goals: Wake up early....,joscchh
1,RT @michael_adler_: Fing gut an! Bonn braucht ...,joscchh
2,RT @EurAsiaBridgeEU: good topic - well done.,joscchh
3,"Gestern haben @Lichtemomente, @askans, @michae...",joscchh


In [49]:
tweets['Handle'].value_counts()

JohnMillerTX      20
lottielane_       20
rosannafanni      20
joscchh           20
christinacrupp     1
Name: Handle, dtype: int64

In [50]:
tweets.rename(columns={0: 'Tweets'}, inplace=True)
tweets.head()        

,Tweets,Handle
0,RT @z_edian: Unser @snv_berlin Expert:innenwor...,christinacrupp
0,RT @rachel_groves: Daily goals: Wake up early....,joscchh
1,RT @michael_adler_: Fing gut an! Bonn braucht ...,joscchh
2,RT @EurAsiaBridgeEU: good topic - well done.,joscchh
3,"Gestern haben @Lichtemomente, @askans, @michae...",joscchh


#### What other sort of meta data can I extract from the tweets?
Consider getting date/time stamp and location of each tweet and adding it to the df

#### Extracting other meta data from tweepy internal methods:

Doesn't work on tweets df created, testing below on my own tweets:

In [51]:
def get_user_tweets(api, username):
        """Return a list of all tweets from the authenticated API"""
        tweets = []
        for status in tweepy.Cursor(api.user_timeline, screen_name=username).items():
            tweets.append(status)
        return tweets 

alltweets = get_user_tweets(api, username='kate_saslow')

print("Number of tweets extracted: {}.\n".format(len(alltweets)))

Number of tweets extracted: 181.



In [52]:
testtweets = get_user_tweets(api, username='SirJacobSaslow')

print("Number of tweets extracted: {}.\n".format(len(testtweets)))

Number of tweets extracted: 27.



In [53]:
print("5 recent tweets:\n")
for tweet in alltweets[:5]:
    print(tweet.text)
    print()

5 recent tweets:

RT @Fabian_Reetz: #Job-Alert 📣
You are into #sustainability or #energytransition and know how to code? 

We at @100ProzentEE are looking fo…

@jimenezroy All of them 🙃

@z_edian @realDa_Kir @MZinkanell @HonkHase ich mach gerade einen Remote/Teilzeit/englischen Kurs mit Ironhack Berli… https://t.co/1PKpzdNoad

@noatamir Thanks for this! I have been trying to decide which to use, this is super timely and helpful 🙃

RT @DerenicByrd: Kalief Browder allegedly stole a backpack at 16, spent 3 yrs at Riker's Island without trial.

Riley Williams stole a lapt…



In [54]:
# We create a pandas DataFrame as follows
# Note: We loop through each element and add it to the DataFrame
data = pd.DataFrame(data=[tweet.text for tweet in alltweets], columns=['Tweets'])

# We display the first 10 elements of the DataFrame:
display(data.head(10))

,Tweets
0,RT @Fabian_Reetz: #Job-Alert 📣\nYou are into #...
1,@jimenezroy All of them 🙃
2,@z_edian @realDa_Kir @MZinkanell @HonkHase ich...
3,@noatamir Thanks for this! I have been trying ...
4,RT @DerenicByrd: Kalief Browder allegedly stol...
5,"RT @DPZ_Berlin: Twitter, Facebook &amp; Co. sp..."
6,RT @CoriBush: Squad up. https://t.co/bp2KATL22R
7,RT @snv_berlin: Ein ereignisreiches Jahr in #T...
8,"RT @TspBackgroundDi: Im #Porträt: Pegah Maham,..."
9,RT @nicsigni: both a blessing and curse that m...


In [55]:
print(dir(alltweets[0]))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_api', '_json', 'author', 'contributors', 'coordinates', 'created_at', 'destroy', 'entities', 'favorite', 'favorite_count', 'favorited', 'geo', 'id', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'parse', 'parse_list', 'place', 'retweet', 'retweet_count', 'retweeted', 'retweeted_status', 'retweets', 'source', 'source_url', 'text', 'truncated', 'user']


In [56]:
# Print a selection of attributes from the first Tweet
print(alltweets[0].id)
print(alltweets[0].created_at)
print(alltweets[0].source)
print(alltweets[0].favorite_count)
print(alltweets[0].retweet_count)
print(alltweets[0].geo)
print(alltweets[0].coordinates)
print(alltweets[0].entities)

1358834091836194821
2021-02-08 17:44:28
Twitter Web App
0
5
None
None
{'hashtags': [{'text': 'Job', 'indices': [18, 22]}, {'text': 'sustainability', 'indices': [44, 59]}, {'text': 'energytransition', 'indices': [63, 80]}], 'symbols': [], 'user_mentions': [{'screen_name': 'Fabian_Reetz', 'name': 'Fabian Reetz', 'id': 768089137291333632, 'id_str': '768089137291333632', 'indices': [3, 16]}, {'screen_name': '100ProzentEE', 'name': '100pes', 'id': 29702898, 'id_str': '29702898', 'indices': [111, 124]}], 'urls': []}


In [57]:
print(testtweets[0].id)
print(testtweets[0].created_at)
print(testtweets[0].source)
print(testtweets[0].favorite_count)
print(testtweets[0].retweet_count)
print(testtweets[0].geo)
print(testtweets[0].coordinates)
print(testtweets[0].entities)

1181903719803301890
2019-10-09 12:06:20
Twitter for iPhone
0
51896
None
None
{'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'BillKristol', 'name': 'Bill Kristol', 'id': 2800581040, 'id_str': '2800581040', 'indices': [3, 15]}], 'urls': []}


In [58]:
import numpy as np
# Add attributes of interest
data['len']  = np.array([len(tweet.text) for tweet in alltweets])
data['ID']   = np.array([tweet.id for tweet in alltweets])
data['Date'] = np.array([tweet.created_at for tweet in alltweets])
data['geo'] = np.array([tweet.geo for tweet in alltweets])
data['Likes']  = np.array([tweet.favorite_count for tweet in alltweets])
data['RTs']    = np.array([tweet.retweet_count for tweet in alltweets])

# Display of first 10 elements from DataFrame
display(data.head(10))

,Tweets,len,ID,Date,geo,Likes,RTs
0,RT @Fabian_Reetz: #Job-Alert 📣\nYou are into #...,140,1358834091836194821,2021-02-08 17:44:28,None,0,5
1,@jimenezroy All of them 🙃,25,1358684860269940741,2021-02-08 07:51:28,None,1,0
2,@z_edian @realDa_Kir @MZinkanell @HonkHase ich...,140,1352976222314180608,2021-01-23 13:47:23,None,2,0
3,@noatamir Thanks for this! I have been trying ...,104,1352973752011776000,2021-01-23 13:37:34,None,1,0
4,RT @DerenicByrd: Kalief Browder allegedly stol...,140,1352616684662771713,2021-01-22 13:58:42,None,0,195493
5,"RT @DPZ_Berlin: Twitter, Facebook &amp; Co. sp...",144,1349280755109785608,2021-01-13 09:02:55,None,0,6
6,RT @CoriBush: Squad up. https://t.co/bp2KATL22R,47,1345808234666463237,2021-01-03 19:04:21,None,0,12969
7,RT @snv_berlin: Ein ereignisreiches Jahr in #T...,140,1339912031613415428,2020-12-18 12:34:57,None,0,8
8,"RT @TspBackgroundDi: Im #Porträt: Pegah Maham,...",140,1339556304294207496,2020-12-17 13:01:25,None,0,5
9,RT @nicsigni: both a blessing and curse that m...,118,1338530120450117633,2020-12-14 17:03:44,None,0,17


In [59]:
testdf = pd.DataFrame(data=[tweet.text for tweet in testtweets], columns=['Tweets'])

testdf['len']  = np.array([len(tweet.text) for tweet in testtweets])
testdf['ID']   = np.array([tweet.id for tweet in testtweets])
testdf['Date'] = np.array([tweet.created_at for tweet in testtweets])
testdf['geo'] = np.array([tweet.geo for tweet in testtweets])
testdf['Likes']  = np.array([tweet.favorite_count for tweet in testtweets])
testdf['RTs']    = np.array([tweet.retweet_count for tweet in testtweets])

# Display of first 10 elements from DataFrame
display(testdf.head(10))

,Tweets,len,ID,Date,geo,Likes,RTs
0,RT @BillKristol: Lindsey Graham: A president w...,140,1181903719803301890,2019-10-09 12:06:20,None,0,51896
1,RT @DetroitPistons: Reggie Jackson #NBAVote \n...,84,687340078138552320,2016-01-13 18:26:59,None,0,333
2,#NBAVote Drummond https://t.co/qkKDCdy69P,42,684585456919433216,2016-01-06 04:01:06,None,0,1
3,RT @espn: That's ten straight wins for the @Ti...,118,364925099461591040,2013-08-07 01:44:58,None,0,585
4,“@espn: College Football: Approaching fast -&g...,138,363711746172649472,2013-08-03 17:23:32,None,0,0
5,"@RapGenius Welcome to Atlanta remix, heeeyyyyy",46,362590532200964096,2013-07-31 15:08:14,None,0,0
6,RT @SeinfeldToday: George is briefly implicate...,129,360087109337485312,2013-07-24 17:20:31,None,0,1988
7,RT @espn: Miguel Cabrera is the 1st player in ...,129,355718347704373249,2013-07-12 16:00:37,None,0,2412
8,RT @JustinVerlander: #BackBenoit #BackBenoit #...,92,355352968603774977,2013-07-11 15:48:44,None,0,2256
9,RT @JustinVerlander: RT &amp; help me #BackBen...,144,355352926258069505,2013-07-11 15:48:34,None,0,5098


Now: before applying the for loop to the entire list of handles in the friends_df, consider what other sort of meta data I want from the tweets. Do I also want the date/time stamp? GEOCODE? etc. This way, I am only making one call to the API and getting all the meta data in one fell swoop.

In [60]:
# https://github.com/RodolfoFerro/pandas_twitter/blob/master/01-extracting-data.md

#### Okay, got it to work by entering a specific username. How can I have it iterate through my handles list?

In [61]:
# write new function that gets more data, not just .text
# see internal methods of tweepy for other meta data: https://github.com/RodolfoFerro/pandas_twitter/blob/master/01-extracting-data.md
# same link as used below!

def get_tweet_meta_data(handle):
    """ This function gets not only tweets.text but also other useful meta data using tweepy's internal method.
    Not sure I will need the other data, but this function will generate the important data in lists."""
    
    # extract 200 tweets
    number_tweets = 200
    tweets = api.user_timeline(screen_name=handle)
    
    # Empty array
    texts = []
    ids = []
    timestamps = []
    geocodes = []
    
    # create array of tweet info: username, tweet id, date/time, text, location
    tweets_text_for_csv = [tweet.text for tweet in tweets]
    tweets_id_for_csv = [tweet.id for tweet in tweets]
    tweets_createdat_for_csv = [tweet.created_at for tweet in tweets]
    tweets_geo_for_csv = [tweet.geo for tweet in tweets]
    
    for i in tweets_text_for_csv:
        texts.append(i)
        
    for j in tweets_id_for_csv:
        ids.append(j)
        
    for k in tweets_createdat_for_csv:
        timestamps.append(k)
        
    for l in tweets_geo_for_csv:
        geocodes.append(l)
        
    return texts, ids, timestamps, geocodes

In [ ]:
get_tweet_meta_data('SirJacobSaslow')

In [63]:
# Now try and add to the function to add the lists into a df:

def test_get_tweet_meta_data_to_df(handle):
    """ This function (TEST function for only 5 handles!) gets not only tweets.text but also other useful meta data using tweepy's internal method.
    Not sure I will need the other data, but this function will add it to a DataFrame in one step."""
# extract 200 tweets
    number_tweets = 200
    tweets = api.user_timeline(screen_name=handle)
    tweets_meta_df = pd.DataFrame()
    
    
    # Empty array
    texts = []
    ids = []
    timestamps = []
    geocodes = []
    
    # create array of tweet info: username, tweet id, date/time, text, location
    tweets_text_for_csv = [tweet.text for tweet in tweets]
    tweets_id_for_csv = [tweet.id for tweet in tweets]
    tweets_createdat_for_csv = [tweet.created_at for tweet in tweets]
    tweets_geo_for_csv = [tweet.geo for tweet in tweets]
    
    for i in tweets_text_for_csv:
        texts.append(i)
        
    for j in tweets_id_for_csv:
        ids.append(j)
        
    for k in tweets_createdat_for_csv:
        timestamps.append(k)
        
    for l in tweets_geo_for_csv:
        geocodes.append(l)
    
    for handle in handles[0:5]:
        twts_df = pd.DataFrame(get_tweets(handle))
        twts_df['Handle'] = handle
        twts_df['ID'] = pd.Series(ids)
        twts_df['CreatedAt'] = pd.Series(timestamps)
        twts_df['Location'] = pd.Series(geocodes)
        tweets_meta_df = pd.concat([tweets_meta_df, twts_df], axis=0, ignore_index=True)
    
    return tweets_meta_df

In [64]:
test_tweets = test_get_tweet_meta_data_to_df(handle)

In [65]:
test_tweets.shape

(81, 5)

#### After extracting tweets for all handles in Friends:
After applying the function to all handles (from the Followers df), merge dataframes with tweets.text and followers (merge on handle), to get the location as well. 

Then, clean and standardize how location is written (by country) to use for analysis.

#### Change the `[handles]` to extract tweets for FOLLOWERS, not friends to extract more tweets:

Use the Try/Except clause in function because the old function kept returning a "Not Authorized" error

In [66]:
def get_tweet_meta_data_to_df(handle):
    """ This function takes the test function above and applies it to all handles within the friends_df and gets not only tweets.text but also other useful meta data using tweepy's internal method.
    Not sure I will need the other data, but this function will add it to a DataFrame in one step."""

#     # extract 200 tweets
#     number_tweets = 200
    tweets = api.user_timeline(screen_name=handle)
    tweets_meta_df = pd.DataFrame()
    
    
    # Empty array
    texts = []
    ids = []
    timestamps = []
    geocodes = []
    errors = []
    
    # create array of tweet info: username, tweet id, date/time, text, location
    tweets_text_for_csv = [tweet.text for tweet in tweets]
    tweets_id_for_csv = [tweet.id for tweet in tweets]
    tweets_createdat_for_csv = [tweet.created_at for tweet in tweets]
    tweets_geo_for_csv = [tweet.geo for tweet in tweets]
    
    for i in tweets_text_for_csv:
        texts.append(i)
        
    for j in tweets_id_for_csv:
        ids.append(j)
        
    for k in tweets_createdat_for_csv:
        timestamps.append(k)
        
    for l in tweets_geo_for_csv:
        geocodes.append(l)
    
    for handle in handles:
        try:
            twts_df = pd.DataFrame(get_tweets(handle))
            twts_df['Handle'] = handle
            twts_df['ID'] = pd.Series(ids)
            twts_df['CreatedAt'] = pd.Series(timestamps)
            twts_df['Location'] = pd.Series(geocodes)
            tweets_meta_df = pd.concat([tweets_meta_df, twts_df], axis=0, ignore_index=True)
        except:
            errors.append(handle)
            print('causes error: ', errors)
            
    return tweets_meta_df

In [67]:
df = get_tweet_meta_data_to_df(handle)

causes error:  ['mobilistig']
causes error:  ['mobilistig', 'ayhangucuyener']
causes error:  ['mobilistig', 'ayhangucuyener', 'Belsaza']
causes error:  ['mobilistig', 'ayhangucuyener', 'Belsaza', 'the_diplomark']
causes error:  ['mobilistig', 'ayhangucuyener', 'Belsaza', 'the_diplomark', 'Stephen56906745']
causes error:  ['mobilistig', 'ayhangucuyener', 'Belsaza', 'the_diplomark', 'Stephen56906745', 'keiam']
causes error:  ['mobilistig', 'ayhangucuyener', 'Belsaza', 'the_diplomark', 'Stephen56906745', 'keiam', 'rudizee']
causes error:  ['mobilistig', 'ayhangucuyener', 'Belsaza', 'the_diplomark', 'Stephen56906745', 'keiam', 'rudizee', 'christodros']
causes error:  ['mobilistig', 'ayhangucuyener', 'Belsaza', 'the_diplomark', 'Stephen56906745', 'keiam', 'rudizee', 'christodros', 'NWarnecke']
causes error:  ['mobilistig', 'ayhangucuyener', 'Belsaza', 'the_diplomark', 'Stephen56906745', 'keiam', 'rudizee', 'christodros', 'NWarnecke', 'ricardawgnr']
causes error:  ['mobilistig', 'ayhangucuye

In [21]:
# Tweak the function and use a try and except clause! But make sure you are storing the tweets already inside the function, not at the very end of the function.
# This will theoretically allow you to extract the maximum amount and not just automatically break the code. 

In [68]:
df.shape

(10953, 5)

# Step 3:
### Save tweets as dataframe to use in later analyses:

#### Save df to csv to use for NLP at a later stage:

In [69]:
# NOTE: now there are 2 dataframes
# one called: 'tweets_from_tweepy_small.csv' with 3982 tweets, 
# and one 'tweets_from_tweepy_large.csv' with 10,953 tweets

# CHANGE TO LARGE:
df.to_csv('tweets_from_tweepy_large.csv')